論文<br>
https://arxiv.org/abs/2111.15666<br>
<br>
GitHub<br>
https://github.com/yuval-alaluf/hyperstyle<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/hyperstyle_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

# 環境セットアップ

In [ ]:
!nvidia-smi

## GithubからCode clone

In [ ]:
%cd /content

# for hyperstyle
!git clone https://github.com/yuval-alaluf/hyperstyle.git

## ライブラリのインストール

In [ ]:
%cd /content/hyperstyle

!pip install --upgrade gdown
!pip install moviepy

# install ninja
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

## ライブラリのインポート

In [ ]:
%cd /content/hyperstyle

import time
import sys
import pprint
from tqdm import tqdm
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms
import os
import gdown
import glob

import imageio
from IPython.display import HTML
from base64 import b64encode

from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip

from notebooks.notebook_utils import Downloader, HYPERSTYLE_PATHS, W_ENCODERS_PATHS, run_alignment
from utils.common import tensor2im
from utils.inference_utils import run_inversion
from utils.domain_adaptation_utils import run_domain_adaptation
from utils.model_utils import load_model, load_generator

SEED = 12
np.random.seed(SEED)

# 学習済みモデルのセットアップ

In [ ]:
%cd /content/hyperstyle
!mkdir pretrained_models

hyper = "./pretrained_models/hyperstyle_ffhq.pt"
w_encoder = "./pretrained_models/faces_w_encoder.pt"

if not os.path.exists(hyper):
  gdown.download('https://drive.google.com/uc?id='+'1C3dEIIH1y8w1-zQMCyx7rDF0ndswSXh4', hyper, quiet=False)
if not os.path.exists(w_encoder):
  gdown.download('https://drive.google.com/uc?id='+'1M-hsL3W_cJKs77xM1mwq2e9-J0_m7rHP', w_encoder, quiet=False)

img_transforms = transforms.Compose([
                                transforms.Resize((256, 256)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

# テスト画像のセットアップ

In [ ]:
%cd /content/hyperstyle
!mkdir input_imgs

# root_dir = "./notebooks/images/animations"
# image_paths = glob.glob(root_dir+"/*.jpg")

!wget -c https://www.pakutaso.com/shared/img/thumb/SAYA160312500I9A3721_TP_V4.jpg -O ./input_imgs/test_1.jpg
!wget -c https://www.pakutaso.com/shared/img/thumb/SAYA072160333_TP_V4.jpg -O ./input_imgs/test_2.jpg
!wget -c https://www.pakutaso.com/shared/img/thumb/saya0I9A4189072170014_TP_V4.jpg -O ./input_imgs/test_3.jpg
!wget -c https://www.pakutaso.com/shared/img/thumb/SAYA160312500I9A3721_TP_V4.jpg -O ./input_imgs/test_4.jpg
!wget -c https://www.pakutaso.com/shared/img/thumb/SAYA0I9A8598_TP_V4.jpg -O ./input_imgs/test_5.jpg
!wget -c https://www.pakutaso.com/shared/img/thumb/akanesaya0I9A3747_TP_V4.jpg -O ./input_imgs/test_6.jpg

root_dir = "./input_imgs"
image_paths = glob.glob(root_dir+"/*.jpg")

# モデルのロード

In [ ]:
%cd /content/hyperstyle

net, opts = load_model(hyper, update_opts={"w_encoder_checkpoint_path": w_encoder})

# Utility関数定義

In [ ]:
# image to mp4
def generate_mp4(out_name, images, kwargs):
  writer = imageio.get_writer(out_name + '.mp4', **kwargs)
  for image in images:
    writer.append_data(image)
  writer.close()

def get_latent_and_weight_deltas(inputs, net, opts):
  opts.resize_outputs = False
  opts.n_iters_per_batch = 5
  with torch.no_grad():
    _, latent, weights_deltas, _ = run_inversion(inputs.to("cuda").float(), net, opts)
  weights_deltas = [w[0] if w is not None else None for w in weights_deltas]
  return latent, weights_deltas

def get_result_from_vecs(vectors_a, vectors_b, weights_deltas_a, weights_deltas_b, alpha):
  results = []
  for i in range(len(vectors_a)):
    with torch.no_grad():
      cur_vec = vectors_b[i] * alpha + vectors_a[i] * (1 - alpha)
      cur_weight_deltas = interpolate_weight_deltas(weights_deltas_a, weights_deltas_b, alpha)
      res = net.decoder(
          [cur_vec],
          weights_deltas=cur_weight_deltas,
          randomize_noise=False,
          input_is_latent=True)[0]
      results.append(res[0])
  return results

def interpolate_weight_deltas(weights_deltas_a, weights_deltas_b, alpha):
  cur_weight_deltas = []
  for weight_idx, w in enumerate(weights_deltas_a):
    if w is not None:
      delta = weights_deltas_b[weight_idx] * alpha + weights_deltas_a[weight_idx] * (1 - alpha)
    else:
      delta = None
    cur_weight_deltas.append(delta)
  return cur_weight_deltas

# face align

In [ ]:
%cd /content/hyperstyle

aligned_image_paths = []
for image_path in image_paths:
  image_name = os.path.splitext(os.path.basename(image_path))[0]
  aligned_image = run_alignment(image_path)
  aligned_path = os.path.join(root_dir, f'{image_name}_aligned.jpg')
  # save the aligned image
  aligned_image.save(aligned_path)
  aligned_image_paths.append(aligned_path)
  # use the save aligned images as our input image paths
  image_paths = aligned_image_paths

# invert

In [ ]:
in_images = []
all_vecs = []
all_weights_deltas = []

resize_amount = (opts.output_size, opts.output_size)

# Inference
for image_path in image_paths:
  original_image = Image.open(image_path)
  original_image = original_image.convert("RGB")
  input_image = img_transforms(original_image)
  # get the weight deltas for each image
  result_vec, weights_deltas = get_latent_and_weight_deltas(input_image.unsqueeze(0), net, opts)
  all_vecs.append([result_vec])
  all_weights_deltas.append(weights_deltas)
  in_images.append(original_image.resize(resize_amount))


# Interpolate & Concat images for animation

In [ ]:
n_transition = 25
SIZE = opts.output_size

images = []
image_paths.append(image_paths[0])
all_vecs.append(all_vecs[0])
all_weights_deltas.append(all_weights_deltas[0])
in_images.append(in_images[0])

for i in range(1, len(image_paths)):
  if i == 0:
    alpha_vals = [0] * 10 + np.linspace(0, 1, n_transition).tolist() + [1] * 5
  else:
    alpha_vals = [0] * 5 + np.linspace(0, 1, n_transition).tolist() + [1] * 5
  for alpha in tqdm(alpha_vals):
    image_a = np.array(in_images[i - 1])
    image_b = np.array(in_images[i])
    image_joint = np.zeros_like(image_a)
    up_to_row = int((SIZE - 1) * alpha)
    if up_to_row > 0:
      image_joint[:(up_to_row + 1), :, :] = image_b[((SIZE - 1) - up_to_row):, :, :]
    if up_to_row < (SIZE - 1):
      image_joint[up_to_row:, :, :] = image_a[:(SIZE - up_to_row), :, :]
    result_image = get_result_from_vecs(
        all_vecs[i - 1], all_vecs[i],
        all_weights_deltas[i - 1], all_weights_deltas[i],alpha)[0]

    output_im = tensor2im(result_image)
    res = np.concatenate([image_joint, np.array(output_im)], axis=1)
    images.append(res)

# show result

In [ ]:
kwargs = {'fps': 15}
save_path = "./output"
os.makedirs(save_path, exist_ok=True)

gif_path = os.path.join(save_path, "face_gif")
generate_mp4(gif_path, images, kwargs)

clip = VideoFileClip(gif_path+".mp4")
clip = resize(clip, height=420)
clip.ipython_display()